In [ ]:
import pandas as pd
import numpy as np
import os
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [ ]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: hand, id: 1553xWPAqzydV2nIRShz4D_h8lCD5WYZu
title: Dog_Human_Classification.ipynb, id: 1nJ9eW4221uGHrtwj_sh9e5joOlzXc2gn
title: cnndh.zip, id: 1Jqg5zazL3f4pEi2xBeIvLpfo01GCQxMY
title: Dog_Human_Classification.ipynb, id: 1oKuEW9BMWXZgwUJD6Ne_gKRINO-ssJNp
title: Dog_Human_Classification (1).ipynb, id: 16CV_goHBg-KccPliy8GLlDvl30ZeCj2f
title: Dog_Human_Classification.ipynb, id: 1Z_E82o3dNBgzjfrSeWynQX_exNNXyXn6
title: shruthi.zip, id: 1dZALEop8dEJTMPCeNtICrZWN3V47VH_j
title: MC assignments ECE-C.zip, id: 1JTOC7c78GPfAKvHRE3uc6f_5Rrq2TeZW
title: modelCNN.ipynb, id: 1zeqYA9wrkdExyUvgIymliPbNnNk2e0XP
title: quizdbms.pdf, id: 1E8qXZxHbmGlNyWtJcK2sacJ_-XovPRYh
title: Inter satellite links2.pptx, id: 14DW7NfEyR4OcnagwsO5pKUFsSyveZeFI
title: ICA lab record109.pdf, id: 1RN5mhgMp3JpehKexFRJvZNa8r5w8exw-
title: 109 ICA lab record.docx, id: 1RIvVZmW5j0Ke6F798QdI6MwSrX-e03Dt
title: 109 ICA lab record.docx, id: 19Szi_4fyyl99Dl0ulqLXWKN12HQSKZG0
title: 109 ICA lab reacord.docx, id: 1We5tWbVZ7F70

In [ ]:
downloaded = drive.CreateFile({'id': '16qBB5M1R81wW_bEtCs4eJg1SVHsQn9q4'})
downloaded.GetContentFile('cnndh.zip')

In [ ]:
!unzip cnndh.zip

Streaming output truncated to the last 5000 lines.
  inflating: Dog_Human/train/human/Leslie_Moonves_0002.jpg  
  inflating: Dog_Human/train/human/Leslie_Wiser_Jr_0001.jpg  
  inflating: Dog_Human/train/human/Lester_Holt_0001.jpg  
  inflating: Dog_Human/train/human/Leszek_Miller_0001.jpg  
  inflating: Dog_Human/train/human/Leszek_Miller_0002.jpg  
  inflating: Dog_Human/train/human/Leszek_Miller_0003.jpg  
  inflating: Dog_Human/train/human/Leticia_Dolera_0001.jpg  
  inflating: Dog_Human/train/human/Leticia_Van_de_Putte_0001.jpg  
  inflating: Dog_Human/train/human/Leuris_Pupo_0001.jpg  
  inflating: Dog_Human/train/human/Lew_Rywin_0001.jpg  
  inflating: Dog_Human/train/human/Lewis_Booth_0001.jpg  
  inflating: Dog_Human/train/human/Li_Changchun_0001.jpg  
  inflating: Dog_Human/train/human/Li_Ka-shing_0001.jpg  
  inflating: Dog_Human/train/human/Li_Peng_0001.jpg  
  inflating: Dog_Human/train/human/Li_Peng_0002.jpg  
  inflating: Dog_Human/train/human/Li_Peng_0003.jpg  
  inflati

In [ ]:
base_model= MobileNet(weights='imagenet',include_top=False)

In [ ]:
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, None, None, 

In [ ]:
CLASSES = 3
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool')(x)
x = tf.keras.layers.Dropout(0.4)(x)
predictions =tf.keras.layers.Dense(3, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
      
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
WIDTH = 299
HEIGHT = 299
BATCH_SIZE = 32
TRAIN_DIR=r'Dog_Human/train'
TEST_DIR=r'Dog_Human/test'
# data prep
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical')
    
validation_generator = validation_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical')
print(train_generator.class_indices)
print(validation_generator.class_indices)

Found 19810 images belonging to 3 classes.
Found 3504 images belonging to 3 classes.
{'dog': 0, 'human': 1, 'nothd': 2}
{'dog': 0, 'human': 1, 'nothd': 2}


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
EPOCHS = 2
BATCH_SIZE = 32
STEPS_PER_EPOCH = 320
VALIDATION_STEPS=64
model.fit_generator(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS
    )

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
320/320 [==============================] - 318s 990ms/step - loss: 0.1900 - accuracy: 0.9276 - val_loss: 0.0951 - val_accuracy: 0.9595
Epoch 2/2
320/320 [==============================] - 310s 969ms/step - loss: 0.0715 - accuracy: 0.9747 - val_loss: 0.0766 - val_accuracy: 0.9736


In [ ]:
from tensorflow.python.keras.preprocessing import image

In [ ]:
img = image.load_img(r'Dog_Human/test/dog/1003.jpg', target_size=(299,299))
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)
pred = model.predict(x)[0]
print(pred)
y_classes = pred.argmax(axis=-1)
y_classes

[9.9219537e-01 5.7447520e-05 7.7471104e-03]


0

In [ ]:
img = image.load_img(r'Dog_Human/test/human/Sharon_Osbourne_0003.jpg', target_size=(299,299))
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)
pred = model.predict(x)[0]
print(pred)
y_classes = np.argmax(pred)
y_classes

[1.6443998e-02 9.8297566e-01 5.8043224e-04]


1

In [ ]:
!pip install keras
!pip install keras.models


  Using cached https://files.pythonhosted.org/packages/25/b9/b97f7202081346923af692be7c926faf7b10c4ed432f99dca416786fec1e/keras_models-0.0.7-py3-none-any.whl


In [ ]:

from keras.models import model_from_json
model_json = model.to_json()

with open("cnn.json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights("cnn.h5")

In [ ]:
train_generator.class_indices

{'dog': 0, 'human': 1, 'nothd': 2}

In [ ]:
df.loc[df['category']>100, 'image_name']

NameError: ignored